In [53]:
import os

from cassandra.cluster import Cluster
from cassandra.query import BatchStatement
from cassandra.auth import PlainTextAuthProvider
from cassandra import WriteTimeout

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import csv

from fredapi import Fred

import pandas as pd

import yfinance as yf
from datetime import datetime, timedelta
import time
from tqdm import tqdm

In [2]:
# Load ticker symbols from nasdaq screener csv file
def load_tickers(filepath):
    df = pd.read_csv(filepath)
    return df.iloc[:, 0].tolist()

In [3]:
# Fetch historical data for a given ticker
def fetch_stock_data(ticker):
    # Calculate the date 5 years ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*5)
    
    # Fetch historical data
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return data

In [ ]:
# Load ticker symbols
tickers = load_tickers('nasdaq_screener.csv')

# Create a subfolder for the financial data
output_dir = 'fin_data'
os.makedirs(output_dir, exist_ok=True)

# Dictionary to hold data for all tickers
all_data = {}

# Loop through each ticker and fetch its data
index = 0
for ticker in tickers:
    try:
        data = fetch_stock_data(ticker)
        all_data[ticker] = data
        # Save data
        data.to_csv(os.path.join(output_dir, f'{ticker}_5_years_data.csv'))
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

In [ ]:
# Define the directory containing the CSV files
source_dir = os.path.expanduser('fin_data/')

# List all files in the directory
files = os.listdir(source_dir)

# Loop through each file
for file in files:
    file_path = os.path.join(source_dir, file)
    
    # Check if the file is a CSV file
    if file.endswith('.csv'):
        try:
            # Load the CSV file
            df = pd.read_csv(file_path)
            
            # Check if the file is empty or contains NA values
            if df.empty or df.isna().any().any():
                os.remove(file_path)  # Remove the file if it's empty or contains NA
                print(f"Deleted '{file}' because it was empty or contained NA values.")
        except Exception as e:
            print(f"Error processing {file}: {e}")

print("Cleanup complete.")


In [2]:
def create_schema(session):
    # CQL to create a keyspace
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS fin_data WITH replication = {
            'class': 'SimpleStrategy', 
            'replication_factor': 1
        }
    """)

    # CQL to use the keyspace
    session.execute("USE fin_data")

    # CQL to create a table
    session.execute("""
        CREATE TABLE IF NOT EXISTS ohlcv (
            symbol text,
            date timestamp,
            open double,
            high double,
            low double,
            close double,
            adj_close double,
            volume bigint,
            PRIMARY KEY (symbol, date)
        )
    """)

In [3]:
def parse_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d')

In [4]:
def execute_with_retries(session, batch, max_retries=5):
    retries = 0
    backoff_time = 2  # Start with 2 seconds backoff time
    while retries < max_retries:
        try:
            session.execute(batch)
            break  # Break the loop if execution is successful
        except WriteTimeout:
            print(f"WriteTimeout: retrying {retries+1}/{max_retries} after {backoff_time}s...")
            time.sleep(backoff_time)  # Sleep for backoff_time seconds before retrying
            backoff_time *= 2  # Double the backoff time for the next retry
            retries += 1
    if retries == max_retries:
        print("Failed to execute batch after several retries.")

In [5]:
def load_data(session, file_path, symbol):
    batch = BatchStatement()
    batch_size = 0  # Track the size of the batch

    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            batch.add(
                """
                INSERT INTO fin_data.ohlcv (symbol, date, open, high, low, close, adj_close, volume)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (symbol, parse_date(row['Date']), float(row['Open']), float(row['High']),
                 float(row['Low']), float(row['Close']), float(row['Adj Close']), int(row['Volume']))
            )
            batch_size += 1
            if batch_size >= 100:  # Execute batch after collecting 50 statements
                execute_with_retries(session, batch)
                batch = BatchStatement()  # Reset batch after execution
                batch_size = 0  # Reset batch size

        if batch_size > 0:  # Ensure any remaining statements are executed
            execute_with_retries(session, batch)

        print(f"Data from {symbol} loaded successfully.")

In [7]:
def drop_table(keyspace_name, table_name):
    """ Drop a table in a given keyspace """
    # Set up connection parameters
    auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')  # Update credentials if needed
    cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)  # Adjust IP if Cassandra is hosted elsewhere
    session = cluster.connect(keyspace_name)  # Connect to the specified keyspace

    # Drop table
    try:
        session.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table {table_name} has been dropped successfully.")
    except Exception as e:
        print(f"An error occurred while dropping the table: {e}")
    finally:
        # Clean up, close the session and cluster connection
        session.shutdown()
        cluster.shutdown()

In [8]:
def query_symbol_data(symbol, start_date, end_date):
    
    cluster = Cluster(['127.0.0.1'])  # Adjust if Cassandra is hosted elsewhere
    session = cluster.connect('fin_data')

    # Prepare the query
    query = """
        SELECT * FROM ohlcv WHERE symbol = %s AND date >= %s AND date <= %s;
    """
    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    # Execute the query
    try:
        rows = session.execute(query, (symbol, start_dt, end_dt))
        for row in rows:
            print(row)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Clean up, close the session and cluster connection
        session.shutdown()
        cluster.shutdown()

In [9]:
def create_table():
    # Connect to Cassandra
    # Adjust the connection settings as needed for your setup
    auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')  # Update with actual credentials
    cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)
    session = cluster.connect()

    # Create the schema
    create_schema(session)

    print("Keyspace and table created successfully.")

    # Clean up
    session.shutdown()
    cluster.shutdown()

In [57]:
create_table()

Keyspace and table created successfully.


In [58]:
cluster = Cluster(['127.0.0.1'])  # Adjust if Cassandra is hosted elsewhere
session = cluster.connect()

i = 0
data_dir = '/home/xca7/Desktop/Storage-Solutions-for-Big-Data-CA1/fin_data'
for filename in tqdm(os.listdir(data_dir)):
    if i > 100:
        break
    i += 1
    if filename.endswith('.csv'):
        symbol = filename.split('_')[0]  # Extract the symbol from the filename
        load_data(session, os.path.join(data_dir, filename), symbol)

session.shutdown()
cluster.shutdown()

  0%|          | 1/6638 [00:00<1:31:24,  1.21it/s]

Data from PEBK loaded successfully.


  0%|          | 2/6638 [00:01<1:28:25,  1.25it/s]

Data from ARR loaded successfully.


  0%|          | 3/6638 [00:01<1:07:15,  1.64it/s]

...

Data from ADUS loaded successfully.


  2%|▏         | 100/6638 [01:00<1:13:07,  1.49it/s]

Data from YUM loaded successfully.


  2%|▏         | 101/6638 [01:01<1:06:11,  1.65it/s]

Data from EMKR loaded successfully.


In [55]:
#drop_table('fin_data', 'ohlcv')

Table ohlcv has been dropped successfully.


In [54]:
def create_spark_session():
    spark = SparkSession.builder \
        .appName("Cassandra Integration Example") \
        .config("spark.cassandra.connection.host", "localhost") \
        .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
        .getOrCreate()
    return spark

In [ ]:
# Initialize Spark session
spark = create_spark_session()

In [26]:
def list_unique_symbols(spark, keyspace, table):
    # Load the data from Cassandra into a DataFrame
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace=keyspace, table=table) \
        .load()
    
    # Select distinct symbols and collect them into a list
    symbols = df.select("symbol").distinct().collect()
    
    # Extract symbols from rows
    symbol_list = [row['symbol'] for row in symbols]
    print("Unique symbols:", symbol_list)
    return symbol_list

In [39]:
keyspace = 'fin_data'
table = 'ohlcv'
list_unique_symbols(spark, keyspace, table)
print()


Unique symbols: ['ARR', 'PEBK', 'HYFM', 'ZURA', 'CLBT', 'GORO', 'ANTE', 'GCTS', 'ALVR', 'SEEL', 'FA']



In [41]:
def query_symbol_data(spark, keyspace, table, symbol, start_date, end_date):
    # Load data from the specified table and keyspace
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace=keyspace, table=table) \
        .load()

    # Filter data for the specified symbol and date range
    filtered_df = df.filter(
        (col("symbol") == symbol) & 
        (col("date") >= start_date) & 
        (col("date") <= end_date)
    )

    # Convert to Pandas DataFrame
    pandas_df = filtered_df.toPandas()
    return pandas_df

In [48]:
# Example usage

keyspace = 'fin_data'
table = 'ohlcv'
symbol = 'PEBK'  # Example symbol
start_date = '2019-01-01'  # Example start date
end_date = '2024-01-31'  # Example end date
    
query_symbol_data(spark, keyspace, table, symbol, start_date, end_date)


,symbol,date,adj_close,close,high,low,open,volume
0,PEBK,2019-04-23 01:00:00,25.188534,27.450001,27.450001,27.309999,27.309999,1600
1,PEBK,2019-04-24 01:00:00,24.885725,27.120001,27.459999,26.750000,27.450001,2400
2,PEBK,2019-04-25 01:00:00,24.968306,27.209999,27.750000,27.209999,27.750000,2300
3,PEBK,2019-04-26 01:00:00,24.922428,27.160000,27.790001,26.900000,27.309999,138100
4,PEBK,2019-04-29 01:00:00,25.142654,27.400000,27.730000,27.070000,27.070000,18200
...,...,...,...,...,...,...,...,...
1198,PEBK,2024-01-25 00:00:00,29.670000,29.670000,29.750000,29.540001,29.650000,3200
1199,PEBK,2024-01-26 00:00:00,29.799999,29.799999,29.959999,29.799999,29.959999,3700
1200,PEBK,2024-01-29 00:00:00,29.670000,29.670000,29.799999,29.500000,29.590000,6600
1201,PEBK,2024-01-30 00:00:00,29.570000,29.570000,29.879999,29.570000,29.610001,1600


In [ ]:
# Lets construct some portfolios and see how they perform!